# TNT Loop-Bridge: Creep Response

## Objectives

- Fit TNT Loop-Bridge model to creep data
- Understand stress-dependent bridge fraction evolution
- Analyze delayed yielding and compliance growth
- Quantify creep timescales and strain accumulation
- Perform Bayesian inference for parameter uncertainty

## Setup

In [ ]:
import os
import sys
import time

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax

import numpy as np
import matplotlib.pyplot as plt
import arviz as az
from scipy.integrate import cumulative_trapezoid

from rheojax.core.jax_config import safe_import_jax
jax, jnp = safe_import_jax()
from rheojax.core.jax_config import verify_float64
verify_float64()

from rheojax.models.tnt import TNTLoopBridge

sys.path.insert(0, os.path.join("..", "utils"))
from tnt_tutorial_utils import (
    load_ml_ikh_flow_curve,
    load_pnas_startup,
    load_laponite_relaxation,
    load_ml_ikh_creep,
    load_epstein_saos,
    load_pnas_laos,
    compute_fit_quality,
    print_convergence_summary,
    print_parameter_comparison,
    save_tnt_results,
    get_tnt_loop_bridge_param_names,
    plot_loop_bridge_fraction,
    plot_bell_nu_sweep,
    compute_maxwell_moduli,
    compute_bell_effective_lifetime,
    print_nu_interpretation,
)

param_names = get_tnt_loop_bridge_param_names()

## Theory: Creep Dynamics

### Physical Picture

Under constant stress:
1. **Elastic response**: Immediate strain gamma_0 = sigma / G_eff
2. **Transient creep**: Bridge fraction evolves, effective modulus changes
3. **Steady flow**: If stress exceeds yield threshold, continuous strain accumulation

### Governing Equations

**Stress Balance (Maxwell backbone):**
```
sigma = f_B * G * gamma_e
d(gamma_e)/dt = gamma_dot - gamma_e / tau_b
```

**Bridge Fraction Evolution:**
```
df_B/dt = (1 - f_B)/tau_a - f_B * exp(nu * gamma_dot * tau_b) / tau_b
```

**Total Strain:**
```
gamma(t) = gamma_e(t) + int_0^t gamma_dot(s) ds
```

**Compliance:**
```
J(t) = gamma(t) / sigma
```

### Stress-Dependent Bridge Fraction

At constant stress, the shear rate adjusts to maintain stress balance:
```
gamma_dot(t) = sigma / (f_B(t) * G * tau_b)
```

Higher stress → higher gamma_dot → stronger Bell detachment → lower f_B → higher gamma_dot (positive feedback)

### Delayed Yielding

If stress is close to the dynamic yield stress, the material may exhibit:
- Initial slow creep (f_B ~ f_B_eq)
- Accelerating creep (f_B decreases)
- Eventual flow (f_B stabilizes at low value)

### Compliance Evolution

Compliance J(t) increases due to:
1. Elastic compliance: J_e = 1 / (f_B * G)
2. Viscous flow: dJ/dt = gamma_dot / sigma

As f_B decreases, J_e increases, accelerating compliance growth.

## Load Creep Data

In [ ]:
time_data, shear_rate, stress_applied = load_ml_ikh_creep(stress_pair_index=0)

# Integrate shear rate to get strain
strain = cumulative_trapezoid(shear_rate, time_data, initial=0)

print(f"Data points: {len(time_data)}")
print(f"Time range: {time_data.min():.2e} - {time_data.max():.2e} s")
print(f"Applied stress: {stress_applied:.2f} Pa")
print(f"Strain range: {strain.min():.4f} - {strain.max():.4f}")
print(f"Final shear rate: {shear_rate[-1]:.4e} 1/s")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Strain vs time
ax1.plot(time_data, strain, 'o', label='Data', markersize=6)
ax1.set_xlabel('Time (s)', fontsize=12)
ax1.set_ylabel('Strain', fontsize=12)
ax1.set_title(f'Creep Data (σ = {stress_applied:.2f} Pa)', fontsize=14)
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# Compliance vs time
compliance = strain / stress_applied
ax2.loglog(time_data, compliance, 'o', label='Data', markersize=6)
ax2.set_xlabel('Time (s)', fontsize=12)
ax2.set_ylabel('Compliance J(t) (1/Pa)', fontsize=12)
ax2.set_title('Creep Compliance', fontsize=14)
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
display(fig)
plt.close(fig)

## NLSQ Fitting

In [ ]:
model = TNTLoopBridge()

print("Starting NLSQ fit...")
t_start = time.time()

nlsq_result = model.fit(time_data, strain, test_mode='creep', stress=stress_applied)

t_nlsq = time.time() - t_start
print(f"\nNLSQ fit completed in {t_nlsq:.2f} seconds")
print(f"\nFitted parameters:")
for name in param_names:
    value = getattr(model, name)
    print(f"  {name}: {value:.4e}")

metrics = compute_fit_quality(time_data, strain, model, 'creep', stress=stress_applied)
print(f"\nFit quality:")
print(f"  R²: {metrics['r_squared']:.6f}")
print(f"  RMSE: {metrics['rmse']:.4e}")
print(f"  Max relative error: {metrics['max_rel_error']:.2f}%")

## NLSQ Fit Visualization

In [ ]:
time_pred = jnp.linspace(time_data.min(), time_data.max(), 200)
strain_pred = model.predict(time_pred, test_mode='creep', stress=stress_applied)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Creep curve
ax1.plot(time_data, strain, 'o', label='Data', markersize=6, alpha=0.7)
ax1.plot(time_pred, strain_pred, '-', label='NLSQ Fit', linewidth=2)
ax1.set_xlabel('Time (s)', fontsize=12)
ax1.set_ylabel('Strain', fontsize=12)
ax1.set_title(f'Creep Fit (R² = {metrics["r_squared"]:.4f})', fontsize=14)
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# Residuals
strain_fit = model.predict(time_data, test_mode='creep', stress=stress_applied)
residuals = (strain - strain_fit) / strain * 100
ax2.plot(time_data, residuals, 'o', markersize=6)
ax2.axhline(0, color='k', linestyle='--', alpha=0.3)
ax2.set_xlabel('Time (s)', fontsize=12)
ax2.set_ylabel('Relative Error (%)', fontsize=12)
ax2.set_title('Fit Residuals', fontsize=14)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
display(fig)
plt.close(fig)

## Physical Analysis: Stress-Dependent Bridge Fraction

In [ ]:
# Estimate effective shear rate from strain growth
strain_rate_pred = jnp.gradient(strain_pred, time_pred)

# Estimate bridge fraction from stress balance
# sigma = f_B * G * gamma_dot * tau_b
# f_B = sigma / (G * gamma_dot * tau_b)
f_B_est = stress_applied / (model.G * strain_rate_pred * model.tau_b + 1e-10)  # avoid division by zero
f_B_est = jnp.clip(f_B_est, 0.0, 1.0)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Strain rate evolution
ax1.semilogy(time_pred, strain_rate_pred, '-', linewidth=2)
ax1.set_xlabel('Time (s)', fontsize=12)
ax1.set_ylabel('Shear Rate (1/s)', fontsize=12)
ax1.set_title('Shear Rate Evolution During Creep', fontsize=14)
ax1.grid(True, alpha=0.3)

# Bridge fraction estimate
ax2.plot(time_pred, f_B_est, '-', linewidth=2)
ax2.axhline(model.f_B_eq, color='r', linestyle='--', alpha=0.5, label=f'f_B_eq = {model.f_B_eq:.4f}')
ax2.set_xlabel('Time (s)', fontsize=12)
ax2.set_ylabel('Bridge Fraction f_B', fontsize=12)
ax2.set_title('Bridge Fraction Evolution (Estimated)', fontsize=14)
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)
ax2.set_ylim([0, 1])

plt.tight_layout()
display(fig)
plt.close(fig)

print(f"\nCreep dynamics:")
print(f"  Initial shear rate: {strain_rate_pred[0]:.4e} 1/s")
print(f"  Final shear rate: {strain_rate_pred[-1]:.4e} 1/s")
print(f"  Acceleration factor: {strain_rate_pred[-1] / strain_rate_pred[0]:.2f}")
print(f"  Estimated initial f_B: {f_B_est[0]:.4f}")
print(f"  Estimated final f_B: {f_B_est[-1]:.4f}")

## Physical Analysis: Compliance Evolution

In [ ]:
# Compliance from model prediction
compliance_pred = strain_pred / stress_applied

# Elastic compliance estimate
J_e = 1.0 / (model.G * model.f_B_eq)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Compliance vs time
ax1.loglog(time_data, compliance, 'o', label='Data', markersize=6, alpha=0.7)
ax1.loglog(time_pred, compliance_pred, '-', label='NLSQ Fit', linewidth=2)
ax1.axhline(J_e, color='r', linestyle='--', alpha=0.5, label=f'J_e = {J_e:.4e} 1/Pa')
ax1.set_xlabel('Time (s)', fontsize=12)
ax1.set_ylabel('Compliance J(t) (1/Pa)', fontsize=12)
ax1.set_title('Creep Compliance', fontsize=14)
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# Compliance growth rate
dJ_dt = jnp.gradient(compliance_pred, time_pred)
ax2.loglog(time_pred[1:], dJ_dt[1:], '-', linewidth=2)
ax2.set_xlabel('Time (s)', fontsize=12)
ax2.set_ylabel('dJ/dt (1/(Pa·s))', fontsize=12)
ax2.set_title('Compliance Growth Rate', fontsize=14)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
display(fig)
plt.close(fig)

print(f"\nCompliance analysis:")
print(f"  Elastic compliance J_e: {J_e:.4e} 1/Pa")
print(f"  Initial compliance: {compliance_pred[0]:.4e} 1/Pa")
print(f"  Final compliance: {compliance_pred[-1]:.4e} 1/Pa")
print(f"  Compliance growth: {compliance_pred[-1] / compliance_pred[0]:.2f}x")

## Physical Analysis: Stress Sweep

In [ ]:
# Simulate creep at different stress levels
stress_sweep = jnp.array([stress_applied * 0.5, stress_applied, stress_applied * 1.5, stress_applied * 2.0])
time_sweep = jnp.linspace(0, time_data.max(), 200)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

for sigma in stress_sweep:
    strain_sweep = model.predict(time_sweep, test_mode='creep', stress=sigma)
    ax1.plot(time_sweep, strain_sweep, '-', linewidth=2, label=f'σ = {sigma:.2f} Pa')

ax1.set_xlabel('Time (s)', fontsize=12)
ax1.set_ylabel('Strain', fontsize=12)
ax1.set_title('Creep Curves at Different Stresses', fontsize=14)
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# Final strain vs stress
final_strains = []
for sigma in stress_sweep:
    strain_sweep = model.predict(time_sweep, test_mode='creep', stress=sigma)
    final_strains.append(strain_sweep[-1])

ax2.plot(stress_sweep, final_strains, 'o-', linewidth=2, markersize=8)
ax2.set_xlabel('Stress (Pa)', fontsize=12)
ax2.set_ylabel('Final Strain', fontsize=12)
ax2.set_title('Final Strain vs Applied Stress', fontsize=14)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
display(fig)
plt.close(fig)

print(f"\nStress dependence:")
for i, sigma in enumerate(stress_sweep):
    print(f"  σ = {sigma:.2f} Pa: final strain = {final_strains[i]:.4f}")

## Bayesian Inference

In [ ]:
NUM_WARMUP = 200
NUM_SAMPLES = 500
NUM_CHAINS = 1

print(f"Starting Bayesian inference with NUTS...")
print(f"  Warmup: {NUM_WARMUP}, Samples: {NUM_SAMPLES}, Chains: {NUM_CHAINS}")

t_start = time.time()
bayes_result = model.fit_bayesian(
    time_data, strain,
    test_mode='creep',
    stress=stress_applied,
    num_warmup=NUM_WARMUP,
    num_samples=NUM_SAMPLES,
    num_chains=NUM_CHAINS,
    seed=42
)
t_bayes = time.time() - t_start

print(f"\nBayesian inference completed in {t_bayes:.2f} seconds")
print(f"Speedup vs NLSQ: {t_bayes/t_nlsq:.1f}x slower (includes MCMC overhead)")

## Convergence Diagnostics

In [ ]:
print_convergence_summary(bayes_result, param_names)

## Parameter Comparison: NLSQ vs Bayesian

In [ ]:
print_parameter_comparison(model, bayes_result.posterior_samples, param_names)

## ArviZ: Trace Plot

In [ ]:
idata = az.from_dict(posterior=bayes_result.posterior_samples)

fig = az.plot_trace(idata, var_names=param_names, compact=False, backend_kwargs={'figsize': (12, 10)})
plt.tight_layout()
display(fig)
plt.close()

## ArviZ: Posterior Distributions

In [ ]:
fig = az.plot_posterior(idata, var_names=param_names, hdi_prob=0.95, backend_kwargs={'figsize': (12, 8)})
plt.tight_layout()
display(fig)
plt.close()

## ArviZ: Pair Plot

In [ ]:
fig = az.plot_pair(
    idata,
    var_names=param_names,
    kind='kde',
    marginals=True,
    backend_kwargs={'figsize': (14, 14)}
)
plt.tight_layout()
display(fig)
plt.close()

## Posterior Predictive

In [ ]:
posterior = bayes_result.posterior_samples
n_draws = 200
indices = np.random.choice(NUM_SAMPLES, size=n_draws, replace=False)

predictions = []
for i in indices:
    params_i = jnp.array([posterior[name][i] for name in param_names])
    pred = model.model_function(jnp.array(time_pred), params_i, test_mode='creep', stress=stress_applied)
    predictions.append(np.array(pred))

predictions = np.array(predictions)
pred_mean = predictions.mean(axis=0)
pred_lower = np.percentile(predictions, 2.5, axis=0)
pred_upper = np.percentile(predictions, 97.5, axis=0)

fig, ax = plt.subplots(figsize=(10, 7))
ax.plot(time_data, strain, 'o', label='Data', markersize=6, alpha=0.7, zorder=3)
ax.plot(time_pred, pred_mean, '-', label='Posterior Mean', linewidth=2, zorder=2)
ax.fill_between(time_pred, pred_lower, pred_upper, alpha=0.3, label='95% Credible Interval', zorder=1)
ax.set_xlabel('Time (s)', fontsize=12)
ax.set_ylabel('Strain', fontsize=12)
ax.set_title('Posterior Predictive Distribution', fontsize=14)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
display(fig)
plt.close(fig)

## Physical Interpretation

In [ ]:
print("\n=== Physical Interpretation ===")
print(f"\n1. Material Properties:")
print(f"   - Plateau modulus G: {model.G:.4e} Pa")
print(f"   - Equilibrium bridge fraction: {model.f_B_eq:.4f}")
print(f"   - Elastic compliance: {J_e:.4e} 1/Pa")

print(f"\n2. Creep Conditions:")
print(f"   - Applied stress: {stress_applied:.4e} Pa")
print(f"   - Test duration: {time_data.max():.4e} s")
print(f"   - Final strain: {strain[-1]:.4f}")
print(f"   - Final compliance: {compliance[-1]:.4e} 1/Pa")

print(f"\n3. Kinetic Timescales:")
print(f"   - Bridge detachment time tau_b: {model.tau_b:.4e} s")
print(f"   - Loop attachment time tau_a: {model.tau_a:.4e} s")
print(f"   - Ratio tau_a/tau_b: {model.tau_a/model.tau_b:.4f}")

print(f"\n4. Flow Behavior:")
print(f"   - Initial shear rate: {strain_rate_pred[0]:.4e} 1/s")
print(f"   - Final shear rate: {strain_rate_pred[-1]:.4e} 1/s")
print(f"   - Acceleration: {strain_rate_pred[-1] / strain_rate_pred[0]:.2f}x")
if strain_rate_pred[-1] > strain_rate_pred[0] * 2:
    print(f"   - Accelerating creep detected (possible delayed yielding)")
else:
    print(f"   - Steady creep behavior")

print(f"\n5. Bridge Depletion:")
print(f"   - Estimated initial f_B: {f_B_est[0]:.4f}")
print(f"   - Estimated final f_B: {f_B_est[-1]:.4f}")
print(f"   - Bridge fraction loss: {(1 - f_B_est[-1]/f_B_est[0])*100:.2f}%")

print(f"\n6. Bell Detachment:")
print(f"   - Nu parameter: {model.nu:.4f}")
gamma_dot_avg = strain[-1] / time_data[-1]
force_factor = jnp.exp(model.nu * gamma_dot_avg * model.tau_b)
print(f"   - Average shear rate: {gamma_dot_avg:.4e} 1/s")
print(f"   - Bell force factor: {force_factor:.4f}")
print(f"   - Detachment enhancement: {force_factor:.2f}x")

## Save Results

In [ ]:
save_tnt_results(model, bayes_result, "loop_bridge", "creep", param_names)
print("Results saved to reference_outputs/tnt/loop_bridge_creep_results.npz")

## Key Takeaways

1. **Stress-Dependent Kinetics**: Higher stress → higher shear rate → stronger Bell detachment → lower f_B

2. **Delayed Yielding**: Material may exhibit slow initial creep followed by acceleration as f_B decreases

3. **Compliance Growth**: J(t) increases due to both elastic (1/G_eff) and viscous (integral of gamma_dot) contributions

4. **Positive Feedback**: Bridge depletion → reduced modulus → higher shear rate → more depletion

5. **Timescale Control**: tau_b and tau_a govern creep rate and recovery potential

6. **Bell Physics**: Force-enhanced detachment (nu > 0) is crucial for stress-dependent creep

7. **Experimental Challenge**: Creep data requires careful integration, numerical noise amplifies in shear rate